# 半监督词分类算法
https://blog.csdn.net/Yellow_python/article/details/100940617

In [1]:
from jieba import cut
from re import fullmatch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from collections import Counter

读语料

In [2]:
with open('x_train.txt', encoding='utf-8') as f:
    x_train = f.read().strip().split("\n")
with open('y_train.txt', encoding='utf-8') as f:
    y_train = f.read().strip().split("\n")
with open('x_test.txt', encoding='utf-8') as f:
    x_test = f.read().strip().split("\n")
with open('y_test.txt', encoding='utf-8') as f:
    y_test = f.read().strip().split("\n")

分词器

In [3]:
match = lambda word: fullmatch('[a-zA-Z\u4e00-\u9fa5]+', word)
stopwords = set('的了很买是都还也我就在这那又里哦和')
tokenizer = lambda text: (word for word in cut(text, HMM=False) if match(word) and word not in stopwords)

向量化

In [4]:
vectorizer = TfidfVectorizer(tokenizer=tokenizer)
X_train = vectorizer.fit_transform(x_train)
X_test = vectorizer.transform(x_test)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\HONGJI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.764 seconds.
Prefix dict has been built succesfully.


分类模型

In [5]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.9138333333333334


可视化函数

In [6]:
yellow = lambda w: '\033[033m{}\033[0m'.format(w)
hot = lambda i: '\033[034m%.2f\033[0m' % i if i < .5 else '\033[031m%.2f\033[0m' % i

### 全监督词分类

In [7]:
words = Counter(word for text in x_train for word in tokenizer(text))
for word, freq in words.most_common(100):
    pred = clf.predict_proba(vectorizer.transform([word]))[0][1]
    print(word, yellow(freq), hot(pred))

苹果 1789 0.61
好 1555 0.81
不 1424 0.25
京东 1088 0.63
吃 867 0.66
有 769 0.39
不错 719 0.94
小 623 0.31
新鲜 595 0.60
水果 549 0.42
一个 546 0.42
好吃 540 0.75
个 509 0.34
差 504 0.06
甜 476 0.72
脆 476 0.90
没有 473 0.54
没 454 0.46
烂 450 0.12
坏 442 0.34
给 442 0.41
太 439 0.23
大 435 0.60
味道 402 0.70
非常 343 0.70
包装 341 0.83
快递 338 0.71
就是 331 0.63
到 320 0.54
说 319 0.29
个头 311 0.67
快 306 0.89
这个 300 0.51
可以 298 0.75
还是 279 0.49
东西 272 0.54
多 262 0.53
挺 257 0.86
再 250 0.54
物流 247 0.80
果 243 0.46
购买 234 0.70
火龙果 232 0.04
吧 229 0.47
收到 224 0.49
第一次 218 0.39
这次 217 0.45
失望 213 0.08
比 212 0.58
满意 207 0.65
上 206 0.47
不是 203 0.50
一次 198 0.32
口感 196 0.72
而且 194 0.28
以后 184 0.38
酸 181 0.26
超市 181 0.57
来 176 0.63
过 174 0.46
知道 173 0.38
感觉 172 0.53
有点 172 0.63
速度 172 0.88
送货 170 0.89
一点 167 0.24
啊 166 0.35
但是 165 0.66
不会 164 0.15
这样 163 0.18
大小 161 0.74
会 158 0.83
还有 158 0.19
真的 157 0.40
比较 157 0.82
评 156 0.12
购物 155 0.40
两个 155 0.20
喜欢 155 0.93
下次 153 0.67
价格 151 0.74
特别 148 0.49
要 148 0.43
不好 147 0.15
人 147 0.41
这么 147 

### 半监督词分类

In [8]:
y_pred = clf.predict_proba(X_test)
c1, c2, c3 = Counter(), Counter(), Counter()
for text, pred in zip(x_test, y_pred):
    for word in cut(text):
        if word not in words and match(word) and word not in stopwords:
            c1[word] += 1
            c2[word] += pred[1]
            c3[word] += 0 if pred[0] > pred[1] else 1
for word, freq in c1.most_common(100):
    print(word, yellow(freq), hot(c2[word] / freq), hot(c3[word] / freq))

差评 216 0.08 0.00
很脆 113 0.89 0.96
还会 106 0.90 0.98
不甜 96 0.29 0.15
脆甜 96 0.88 0.95
这是 81 0.28 0.19
太差 78 0.12 0.00
买过 64 0.58 0.58
不买 64 0.16 0.03
全是 60 0.15 0.03
不脆 60 0.33 0.23
一看 56 0.20 0.11
很差 55 0.16 0.02
给力 55 0.89 0.96
脆脆 54 0.90 0.98
坏果 51 0.53 0.55
一星 48 0.14 0.02
网购 47 0.30 0.23
好用 47 0.53 0.98
还行 40 0.70 0.82
甜脆 40 0.89 0.95
中果 36 0.16 0.06
无语 32 0.10 0.00
大果 31 0.16 0.06
送人 29 0.41 0.34
超快 29 0.92 0.97
冷链 28 0.78 0.86
不大 28 0.55 0.50
点赞 28 0.90 1.00
有坏 27 0.43 0.37
差差 26 0.05 0.00
很足 26 0.90 1.00
个装 24 0.61 0.62
华圣 23 0.75 0.87
看图 22 0.22 0.23
小个 22 0.41 0.32
买到 21 0.27 0.19
不太 21 0.41 0.33
太大 21 0.42 0.29
有个 20 0.49 0.50
挺脆 20 0.90 1.00
偏小 19 0.57 0.58
没好 19 0.26 0.11
赞赞 19 0.95 1.00
总重 18 0.57 0.67
很赞 18 0.88 1.00
有烂 18 0.32 0.11
有图 17 0.22 0.18
没坏 17 0.51 0.47
太坑 16 0.10 0.00
宏辉 16 0.60 0.81
小果 15 0.16 0.07
不说 15 0.13 0.07
太酸 15 0.24 0.13
不像 14 0.47 0.50
佳农 14 0.29 0.29
烂果 14 0.48 0.43
没见 13 0.11 0.08
kg 13 0.46 0.54
果果 13 0.78 0.92
小贵 13 0.77 0.92
很香 13 0.92 1.00
个果 12